In [1]:
import sys
sys.path.insert(0,'/home/lshtanko/Programming/another-fem-framework/')
#sys.path.insert(0,'/Users/marusy/Programming/another-fem-framework/')
#sys.path.insert(0,'/Users/leonshting/Programming/Schlumberger/fem-framework/')

In [2]:
#import h5py

from operator_assembler.assembly_interface import AssemblyInterface2D
from operator_assembler.n_to_1_ddof_allocator import Nto1Allocator2D
from grid.grid_manager import GridManager
from grid.allocator_interface import InterfaceDofAllocator2D
from grid.grid_domain import GridDomain
from operator_assembler.matrix_assembler import MatrixAssembler2D
from scipy.ndimage import imread
from grid import cell, grid_layer, grid_manager
import h5py
import numpy as np

from common import visual
from common.tests import *
from common.helpers import *
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse import linalg

%matplotlib inline

#### Bad cell

In [ ]:
data_start = (16, 20)
data_shape = (4, 4)
data_end = tuple([i+j for i,j in zip(data_start, data_shape)])

h5_data = h5py.File('../../../GM_L3D/600/state/viz_GM_1.h5')['index'][0]
data = h5_data[data_start[0]:data_end[0], data_start[1]:data_end[1]]
#data = imread('/Users/marusy/Programming/model/bhi2_labelled0000.tif')[data_start[0]:data_end[0], data_start[1]:data_end[1]]

grid_domain = GridDomain(integer_size=data_shape, domain_size=(1.,1.))
gm = GridManager()
ifma = InterfaceDofAllocator2D(grid_manager=gm.fit(data=data))

gm.draw_grid()

In [3]:
cell1 = cell.Cell2D(size=(1,1), ll_vertex=(0,0))
cell2 = cell.Cell2D(size=(1,1), ll_vertex=(0,1))

cell3 = cell.Cell2D(size=(2,2), ll_vertex=(1,0))

In [4]:
layer1 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell1, cell2]], layer_number=0, index=[1,1], div_index=[1,1])
layer2 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell3]], layer_number=1, index=[1], div_index=[1])

In [5]:
gm = grid_manager.GridManager(dim=2, method='coarse2to1')
gm.grid_layers = [layer1, layer2]
ifma = InterfaceDofAllocator2D(grid_manager=gm)

In [6]:
ma = Nto1Allocator2D(grid_interface=ifma)
ma._make_ddof_index()
grid_domain = GridDomain(domain_size=(3.,2.), integer_size=(3,2))
grid_domain.make_pointwise_index(ma)

In [7]:
ifma2 = AssemblyInterface2D(allocator=ma)
MA = MatrixAssembler2D(assembly_interface=ifma2, grid_domain=grid_domain)


In [ ]:
MA._gg_matrices

In [8]:
deep = MA.assemble()

[(0, 4), (1, 36), (2, 37), (3, 38), (4, 39), (5, 40), (6, 0), (7, 50), (8, 51), (9, 52), (10, 53), (11, 41), (12, 1), (13, 54), (14, 55), (15, 56), (16, 57), (17, 42), (18, 2), (19, 58), (20, 59), (21, 60), (22, 61), (23, 43), (24, 3), (25, 62), (26, 63), (27, 64), (28, 65), (29, 44), (30, 5), (31, 46), (32, 47), (33, 48), (34, 49), (35, 45)] [(0, 10), (1, 6), (2, 7), (3, 8), (4, 9), (5, 4), (6, 11), (7, 20), (8, 21), (9, 22), (10, 23), (11, 0), (12, 12), (13, 24), (14, 25), (15, 26), (16, 27), (17, 1), (18, 13), (19, 28), (20, 29), (21, 30), (22, 31), (23, 2), (24, 14), (25, 32), (26, 33), (27, 34), (28, 35), (29, 3), (30, 15), (31, 16), (32, 17), (33, 18), (34, 19), (35, 5)]
{0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65} {66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 9

In [ ]:
sine_test = grid_domain.vectorize_function(lambda x,y: np.sin(x+y))
product = grid_domain.devectorize_vector(MA.assembled * sine_test)
init = grid_domain.devectorize_function(lambda x,y: np.sin(x+y))

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_vector(MA.assembled * sine_test),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_function(lambda x,y: np.sin(x+y)),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

In [9]:
def partial_diff_test2(pointnum, point, matrix, pointdict, mass_matrix=None,
                      multiple_dofs_on_point=True):
    x_der = 0
    xx_der = 0
    yy_der = 0
    y_der = 0
    xy_der = 0

    if not multiple_dofs_on_point:
        for k, v in pointdict.items():
            dx = point[0] - k[0]
            dy = point[1] - k[1]

            x_der += dx * matrix[pointnum, v]
            xx_der += matrix[pointnum, v] * dx ** 2 / 2
            y_der += dy * matrix[pointnum, v]
            yy_der += dy ** 2 / 2 * matrix[pointnum, v]
            xy_der += dx * dy * matrix[pointnum, v]
    else:
        for k, vs in pointdict.items():
            for v in vs:
                dx = point[0] - k[0]
                dy = point[1] - k[1]

                x_der += dx * matrix[pointnum, v]
                xx_der += matrix[pointnum, v] * dx ** 2 / 2
                y_der += dy * matrix[pointnum, v]
                yy_der += dy ** 2 / 2 * matrix[pointnum, v]
                xy_der += dx * dy * matrix[pointnum, v]

    if mass_matrix is not None:
        print('Mass coef: {}'.format(mass_matrix[pointnum].sum()))

    print('X derivative: {}'.format(x_der))
    print('Y derivative: {}'.format(y_der))
    print('XX derivative: {}'.format(xx_der))
    print('YY derivative: {}'.format(yy_der))
    print('XY_derivative: {}'.format(xy_der), end='\n\n')
    return (xx_der + yy_der)/2

In [10]:
mm = np.zeros(MA.half_glob.shape[0])
for i in grid_domain.normal_index.keys():
    print(i, end=' ')
    mm[i] = partial_diff_test2(matrix=MA.half_glob, point=grid_domain.normal_index[i], pointnum=i, pointdict=grid_domain.pointwise_index)

0 X derivative: 4.9516783925335705e-06
Y derivative: 3.286937261050202e-06
XX derivative: -0.012615711931482315
YY derivative: -0.012616584784628663
XY_derivative: -1.1276531848151272e-09

1 X derivative: -3.003351352636967e-06
Y derivative: 6.452514450722147e-06
XX derivative: -0.01849587330328156
YY derivative: -0.01849429196780694
XY_derivative: 1.8653711499540262e-09

2 X derivative: 2.223439716112664e-06
Y derivative: 3.065670457042526e-07
XX derivative: -0.018494989599944636
YY derivative: -0.018499381470156184
XY_derivative: -3.2855060504002877e-09

3 X derivative: -1.2258274275207326e-06
Y derivative: -4.4143018021375344e-07
XX derivative: -0.012616432423750159
YY derivative: -0.01258569291249695
XY_derivative: 8.390323239544386e-09

4 X derivative: 0.06666414691935572
Y derivative: 1.5749742559565556e-06
XX derivative: -0.002222161709186672
YY derivative: -0.0022218575968308506
XY_derivative: 3.975002880366965e-10

5 X derivative: -0.06666842651917577
Y derivative: 7.979064583

In [11]:
MM = csr_matrix(np.diag(-1/mm))

In [12]:
for i in sorted([81, 82, 83, 84, 85, 70] + [15, 16, 17, 18, 19, 5, 46, 47, 48, 49, 45]):
    print(i, end=' ')
    partial_diff_test(matrix=MM*MA.half_glob, point=grid_domain.normal_index[i], pointnum=i, pointdict=grid_domain.pointwise_index)

5 X derivative: -29.708825747883324
Y derivative: 0.00035556357203093215
XX derivative: -0.9902194533333452
YY derivative: -1.009780546666859
XY_derivative: -1.2161037500460158e-08

15 X derivative: -29.822684795793624
Y derivative: 29.829789159132954
XX derivative: -0.9939298249436672
YY derivative: -1.0060701750562084
XY_derivative: -5.261038992837485e-09

16 X derivative: -30.101161593885806
Y derivative: -0.0066193899389092505
XX derivative: -1.0034778555493846
YY derivative: -0.9965221444505842
XY_derivative: 1.37415753847255e-10

17 X derivative: -29.92917919899811
Y derivative: 0.0013945645448322505
XX derivative: -0.9976091144975565
YY derivative: -1.0023908855024457
XY_derivative: -8.350862377907642e-10

18 X derivative: -30.132277681138845
Y derivative: -0.0030618288383993386
XX derivative: -1.0043761042650552
YY derivative: -0.9956238957349706
XY_derivative: -3.737012549212395e-09

19 X derivative: -29.946128074872764
Y derivative: 0.004968606433385797
XX derivative: -0.9981

In [ ]:
for i in grid_domain.normal_index.keys():
    print(i, end=' ')
    partial_diff_test(matrix=MA.assembled, point=grid_domain.normal_index[i], pointnum=i, pointdict=grid_domain.pointwise_index)

In [ ]:
MA.I_s2b

In [ ]:
MA.half_glob[2].toarray().flatten()

In [ ]:
deep[0][1].toarray()[[2, 7, 6]]

In [ ]:
deep[0][1][2].toarray()

In [ ]:
deep[0][1][:,16].toarray()

In [ ]:
MA.assembled[2].toarray().flatten()[-9:]

In [ ]:
pipka[2].toarray().flatten()

In [ ]:
partial_diff_test(matrix=deep[0][-1], point=grid_domain.normal_index[2], pointnum=2, pointdict=grid_domain.pointwise_index)

In [ ]:
dist = distributed_eye_easy(ma.get_cell_list_of_ddofs(cell=cell3), axis2shape=MA.assembly_interface.get_ddof_count())

In [ ]:
pipka = dist.T * csr_matrix(MA._get_local_gg_matrix(distribution='lobatto', order=2)) * dist

In [ ]:
mm = np.array(MA.assembled_mass.sum(axis=1)).flatten()

In [ ]:
mm_sp = csr_matrix(np.diag(1/mm))

In [ ]:
op =  mm_sp * MA.assembled 

In [ ]:
grid_domain.normal_index

In [ ]:
MA.assembly_interface.allocator.get_cell_list_of_ddofs(cell2)

In [ ]:
plt.imshow(deep[0][-1].toarray(), interpolation='none')

In [ ]:
f = plt.figure(figsize=(10,10))
plt.imshow(deep[0][1].toarray(), interpolation='none')

In [ ]:
partial_diff_test(matrix=MA.assembled, point=grid_domain.normal_index[85], pointnum=85, pointdict=grid_domain.pointwise_index)

In [ ]:
partial_diff_test(matrix=MA.assembled, point=grid_domain.normal_index[19], pointnum=19, pointdict=grid_domain.pointwise_index)

In [ ]:
partial_diff_test(matrix=MA.half_glob, point=grid_domain.normal_index[85], pointnum=85, pointdict=grid_domain.pointwise_index)

In [ ]:
partial_diff_test(matrix=MA.half_glob, point=grid_domain.normal_index[19], pointnum=19, pointdict=grid_domain.pointwise_index)

In [ ]:
csr_matrix(MA._get_local_gg_matrix(distribution='lobatto', order=2))

In [ ]:
from common import polynom_factory

In [ ]:
mmatrix = polynom_factory.local_funcfunc_matrix(distribution='globatto', order=2, dim=2, size=(0,1))

In [ ]:
tomul = np.diag(1/mmatrix.sum(axis=1))

In [ ]:
matrix, index = polynom_factory.local_gradgrad_matrix(distribution='globatto', order=2, dim=2)

In [ ]:
for point, num in index.items():
    partial_diff_test(matrix=np.dot(tomul, matrix), point=point, pointnum=num, pointdict=index, multiple_dofs_on_point=False)

In [ ]:
for point, num in index.items():
    partial_diff_test(matrix=matrix, point=point, pointnum=num, pointdict=index, multiple_dofs_on_point=False)

In [14]:
deep[0][1][5].toarray()

array([[ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.5     ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      , -2.243135,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.     